Abalone Data

In [1]:
import tensorflow as tf
import os
import cv2
import imghdr
import numpy as np
from matplotlib import pyplot as plt
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from sklearn.manifold import TSNE
from sklearn.preprocessing import StandardScaler 
from sklearn.decomposition import PCA
import pandas as pd
import seaborn as sns
from scipy.io import arff
from sklearn.model_selection import train_test_split

In [2]:
data = arff.loadarff('abalone.arff')
df = pd.DataFrame(data[0])
print(df.shape)

df.head()

(4177, 11)


,Sex=M,Sex=F,Sex=I,Length,Diameter,Height,Whole weight,Shucked weight,Viscera weight,Shell weight,Class_Rings
0,1.0,0.0,0.0,0.513514,0.521008,0.084071,0.181335,0.150303,0.132324,0.147982,0.500000
1,1.0,0.0,0.0,0.371622,0.352941,0.079646,0.079157,0.066241,0.063199,0.068261,0.214286
2,0.0,1.0,0.0,0.614865,0.613445,0.119469,0.239065,0.171822,0.185648,0.207773,0.285714
3,1.0,0.0,0.0,0.493243,0.521008,0.110619,0.182044,0.144250,0.149440,0.152965,0.321429
4,0.0,0.0,1.0,0.344595,0.336134,0.070796,0.071897,0.059516,0.051350,0.053313,0.214286


In [3]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
Sex=M,4177.0,0.365813,0.481715,0.0,0.000000,0.000000,1.000000,1.0
Sex=F,4177.0,0.312904,0.463731,0.0,0.000000,0.000000,1.000000,1.0
Sex=I,4177.0,0.321283,0.467025,0.0,0.000000,0.000000,1.000000,1.0
Length,4177.0,0.606746,0.162288,0.0,0.506757,0.635135,0.729730,1.0
Diameter,4177.0,0.593078,0.166790,0.0,0.495798,0.621849,0.714286,1.0
Height,4177.0,0.123466,0.037015,0.0,0.101770,0.123894,0.146018,1.0
Whole weight,4177.0,0.292808,0.173681,0.0,0.155658,0.282451,0.407650,1.0
Shucked weight,4177.0,0.241000,0.149269,0.0,0.124412,0.225286,0.336920,1.0
Viscera weight,4177.0,0.237121,0.144324,0.0,0.122449,0.224490,0.332456,1.0
Shell weight,4177.0,0.236503,0.138717,0.0,0.128052,0.231689,0.326358,1.0


In [4]:
print(df["Class_Rings"])

0       0.500000
1       0.214286
2       0.285714
3       0.321429
4       0.214286
          ...   
4172    0.357143
4173    0.321429
4174    0.285714
4175    0.321429
4176    0.392857
Name: Class_Rings, Length: 4177, dtype: float64


In [5]:
X = df.drop("Class_Rings",axis=1)
X.head()
features = (X.shape)[1] # feature value
print(features) 

10


In [6]:
Y = df['Class_Rings']
Y.head()

0    0.500000
1    0.214286
2    0.285714
3    0.321429
4    0.214286
Name: Class_Rings, dtype: float64

In [7]:
# Train - Test ayrımını yapmak lazım.
X_train_full,X_test,y_train_full,y_test = train_test_split(X,Y,test_size = 0.2)

In [8]:
X_train,X_valid,y_train,y_valid = train_test_split(X_train_full,y_train_full,test_size = 0.2)   

In [9]:
# Model
# 1 Hidden katmanlı yapı

model = Sequential()
model.add(Dense(5, activation='relu')) # Hidden layer
model.add(Dense(1)) # Output layer


In [10]:
# Adam optimizer ile MAE performans kriteri ile
model.compile(tf.keras.optimizers.Adam(), loss='mean_squared_error', metrics=['accuracy'])

In [11]:
history = model.fit(X_train,y_train, epochs=30, batch_size=64,validation_data=(X_valid,y_valid))

Epoch 1/30
42/42 [==============================] - 1s 9ms/step - loss: 0.0189 - accuracy: 3.7425e-04 - val_loss: 0.0122 - val_accuracy: 0.0000e+00
Epoch 2/30
42/42 [==============================] - 0s 5ms/step - loss: 0.0118 - accuracy: 3.7425e-04 - val_loss: 0.0092 - val_accuracy: 0.0000e+00
Epoch 3/30
42/42 [==============================] - 0s 5ms/step - loss: 0.0103 - accuracy: 3.7425e-04 - val_loss: 0.0089 - val_accuracy: 0.0000e+00
Epoch 4/30
42/42 [==============================] - 0s 3ms/step - loss: 0.0100 - accuracy: 3.7425e-04 - val_loss: 0.0087 - val_accuracy: 0.0000e+00
Epoch 5/30
42/42 [==============================] - 0s 3ms/step - loss: 0.0096 - accuracy: 3.7425e-04 - val_loss: 0.0085 - val_accuracy: 0.0000e+00
Epoch 6/30
42/42 [==============================] - 0s 3ms/step - loss: 0.0093 - accuracy: 3.7425e-04 - val_loss: 0.0082 - val_accuracy: 0.0000e+00
Epoch 7/30
42/42 [==============================] - 0s 3ms/step - loss: 0.0091 - accuracy: 3.7425e-04 - val_loss

In [12]:
from scikeras.wrappers import KerasRegressor

In [13]:
keras_clf = KerasRegressor(model = model, optimizer="adam", epochs=50,batch_size=64,loss='mean_squared_error',verbose=0)

In [14]:
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import AdaBoostRegressor

In [15]:
bag_reg = BaggingRegressor(base_estimator=keras_clf,n_estimators=10,random_state=1)

In [16]:
ada_reg = AdaBoostRegressor(base_estimator=keras_clf,n_estimators=10,random_state=1)

In [17]:
ranSub_reg = BaggingRegressor(base_estimator=keras_clf,n_estimators=10,bootstrap=False,bootstrap_features=True,random_state=1)

In [18]:
from mlxtend.evaluate import paired_ttest_5x2cv

In [19]:
t1, p1 = paired_ttest_5x2cv(estimator1=bag_reg,
                              estimator2=ada_reg,
                              X=X, y=Y,
                              random_seed=1)
                            
if p1<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpa_dbv7rp\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpknleoftx\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpfksjeqqh\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp6ac9pp3w\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp42m5zf6g\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpsrj27erf\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpae3w6kfn\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp_yex7nvw\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpsq4tvhrm\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmptmk9a8_t\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp68d5te5o\assets
INFO:tensorflow:Assets written t

In [20]:
t2, p2 = paired_ttest_5x2cv(estimator1=bag_reg,
                              estimator2=ranSub_reg,
                              X=X, y=Y,
                              random_seed=1)
                            
if p2<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmphd9tfef_\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp970rjwda\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpn6fsww2y\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpoxlb24eo\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpukpsgu1t\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpd9m3zyci\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpkweg2ja4\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp0dls1l8m\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp5etgg3vg\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpo2w_ebqf\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpstdtvc51\assets
INFO:tensorflow:Assets written t

In [21]:
t3, p3 = paired_ttest_5x2cv(estimator1=bag_reg,
                              estimator2=model,
                              X=X, y=Y,
                              random_seed=1)
                            
if p3<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpattz29_o\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpjvg7vxk8\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp7vnluyfs\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmphcilgg2x\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpfo6oao91\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpyn8cv6d6\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpg5nec3ap\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpzaskruqo\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpxko54k63\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp_7r08eqy\assets
66/66 [==============================] - 0s 1ms/step
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Tem

In [25]:
t4, p4 = paired_ttest_5x2cv(estimator1=ada_reg,
                              estimator2=ranSub_reg,
                              X=X, y=Y,
                              random_seed=1)
                            
if p4<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpddfk_smd\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp2m0zfnnr\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp44qiaxgg\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp5tvcl8mf\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpa5fw1bv6\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpwuu4thtz\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp102cey2f\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpgpmo4e3o\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpk0e31vq4\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp2d5v_xz6\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp7kxh2l2d\assets
INFO:tensorflow:Assets written t

In [23]:
t5, p5 = paired_ttest_5x2cv(estimator1=ada_reg,
                              estimator2=model,
                              X=X, y=Y,
                              random_seed=1)
                            
if p5<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpbzl9fuon\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpeszcfk78\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpx3n7jwnj\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpadenmxw9\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp8curzjkd\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpyqfs074h\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmps4dpv9hu\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp3srb5z4y\assets
66/66 [==============================] - 0s 1ms/step
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpl_am6239\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmponzt9awk\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Tem

In [24]:
t6, p6 = paired_ttest_5x2cv(estimator1=ranSub_reg,
                              estimator2=model,
                              X=X, y=Y,
                              random_seed=1)
                            
if p6<0.05:
    print("We are rejecting null hypothesis.") # Two model different
else:
    print("We are accepting null hypothesis.") # Two model same

INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpk4ie6bkc\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpukous629\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpykezdonp\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpcaos8rr6\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpho1yxtme\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp_tlbc0x8\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpq9444vlo\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmpqf68wha5\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp0es3wbfs\assets
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Temp\tmp8etpzn3v\assets
66/66 [==============================] - 0s 1ms/step
INFO:tensorflow:Assets written to: C:\Users\enes_\AppData\Local\Tem